In [116]:
from commonfunctions import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
def load_image(file_path):
    image = io.imread(file_path)
    return image

In [125]:

image = load_image("Grades_Sheet_Dataset/12.jpg")

#Read image
img = io.imread("Grades_Sheet_Dataset/12.jpg")

# Rotate 90 degrees clockwise
rotated = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

show_images([rotated])

NameError: name 'cv2' is not defined

In [118]:
from skimage.transform import hough_line, hough_line_peaks,probabilistic_hough_line, rotate
from skimage.transform import ProjectiveTransform, warp
from skimage.measure import find_contours


def order_points(pts):
    rectangle=np.zeros((4,2), dtype="float32")
    
    s=pts.sum(axis=1) # the top left 3andaha smallest x+y w bottom rught akbar
    rectangle[0]=pts[np.argmin(s)]
    rectangle[2]=pts[np.argmax(s)]
    
    difference=np.diff(pts,axis=1) # top right 3andaha smallest y-x w bottom right akbar
    rectangle[1]=pts[np.argmin(difference)]
    rectangle[3]=pts[np.argmax(difference)]
    
    return rectangle


def four_point(image,pts):
    rectangle=order_points(pts)
    
    top_left,top_right,bottom_right,bottom_left=rectangle
    
    bottom_edge_width=np.linalg.norm(bottom_right-bottom_left)
    top_edge_width=np.linalg.norm(top_right-top_left)
    maxwidth=int(max(bottom_edge_width,top_edge_width))# 34an amna3 hetta cropped)

    right_edge_height=np.linalg.norm(top_right-bottom_right)
    left_edge_height=np.linalg.norm(top_left-bottom_left)
    maxheight=int(max(right_edge_height,left_edge_height))
    
    final_rectangle=np.array([[0,0],[maxwidth-1,0],[maxwidth-1,maxheight-1],[0,maxheight-1] ],dtype="float32")
    
    mapping=cv2.getPerspectiveTransform(rectangle,final_rectangle)
    
    result=cv2.warpPerspective(image,mapping,(maxwidth,maxheight))
    
    return result

def detect_document_contour(image):
    gray=rgb2gray(image)
    gray_blurred=gaussian(gray,sigma=1)
    edges=canny(gray_blurred,sigma=1,low_threshold=50,high_threshold=150)
    contours, _ = cv2.findContours(edges,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours[:10]:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            return approx.reshape(4, 2)

    return None
    
    
    
    

def deskew(image):
    result=np.zeros_like(image)
    edge=canny(image,sigma=1,low_threshold=10,high_threshold=70)
    lines=probabilistic_hough_line(edge,line_length=80,line_gap=5)
  
    
    if not lines:
        return image
    
   

    
    #Storing angles of lines detected
    angles=[]
    
    for(x0,y0),(x1,y1) in lines:
        delta_x=x1-x0
        delta_y=y1-y0
        ang=np.degrees(np.arctan2(delta_y,delta_x))
        
        while ang>90:
            ang-=180
           
        while ang<-90:
            ang+=180
            
        angles.append(ang)
        
    horizontal=[a for a in angles if abs(a)<45] #to keep the sllightly tilted rows 
    if len(horizontal)<3:
        return image  
        
    skew=float(np.median(horizontal)) #to avoid extreme outlier eno ybawaz el angles ely tal3a
    skew = float(np.clip(skew, -10, 10))  # prevent over-rotation

    rotated =rotate(image,angle=-skew,resize=False, preserve_range=True )
    return rotated 

            
def preprocessing(image):
    doc_cont=detect_document_contour(image)
    warped=four_point(image,doc_cont)
    
    image_gray = rgb2gray(image)
    #image_blurred = gaussian(image_gray, sigma=1)
    
     #Hsitogram Equalization for uneven lighting / shadows
     #image_clashed=equalize_hist(image_blurred)

    image_deskewed=deskew(image_gray)
  
    return image_deskewed



In [8]:
def cell_extraction(image):
    image = 0

In [7]:
def read_cell(cell_image):
    image = 0

In [6]:
def export_excel(data, file_name):
    df = pd.DataFrame(data)
    df.to_excel(file_name, index=False)

In [121]:
image = load_image("Grades_Sheet_Dataset/12.jpg")
preprocessed_image = preprocessing(image)
show_images([image, preprocessed_image], titles=["Original Image", "Preprocessed Image"])

NameError: name 'cv2' is not defined